In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install python-dotenv vertexai requests


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 20.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
  Attempting uninstall: cachetools
    Found existing installation: cachetools 6.2.1
    Uninstalling cachetools-6.2.1:
      Successfully uninstalled cachetools-6.2.1
  Attempting uninstall: google-cloud-storage
    Found existing installation: google-cloud-storage 3.5.0
    Uninstalling google-cloud-storage-3.5.0:
      Successfully uninstalled google-cloud-storage-3.5.0
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.125.0
    Uninstalling google-cloud-aiplatform-1.125.0:
      Successfully uninstalled google-cloud-aiplatform-1.125

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

# OPTIONAL: If .env does not work on Kaggle, you can set keys manually like this:
# os.environ["SERPAPI_KEY"] = "your_serpapi_key"
# os.environ["AIRTABLE_TOKEN"] = "your_airtable_token"
# os.environ["AIRTABLE_BASE_ID"] = "your_airtable_base_id"
# os.environ["ZAPIER_WEBHOOK_URL"] = "your_webhook_url"


False

In [4]:
import os
import requests

def search_company_profile(query):
    params = {
        "q": query,
        "api_key": os.getenv("SERPAPI_KEY"),
        "engine": "google"
    }
    response = requests.get("https://serpapi.com/search", params=params)
    results = response.json()
    return results.get("organic_results", [])


In [5]:
import os
import requests

def update_lead_record(profile):
    headers = {
        "Authorization": f"Bearer {os.getenv('AIRTABLE_TOKEN')}",
        "Content-Type": "application/json"
    }
    data = {
        "fields": {
            "Name": profile.get("name"),
            "Company": profile.get("company"),
            "Website": profile.get("url"),
            "Notes": profile.get("snippet")
        }
    }
    url = f"https://api.airtable.com/v0/{os.getenv('AIRTABLE_BASE_ID')}/Leads"
    response = requests.post(url, json=data, headers=headers)
    return response.status_code


In [6]:
import os
import requests

def schedule_social_post(post_text, scheduled_time):
    """
    Uses Zapier Webhook (or similar) to schedule a social post.
    For the Kaggle demo, this will TRY to call the webhook.
    """
    webhook_url = os.getenv("ZAPIER_WEBHOOK_URL")
    if not webhook_url:
        # Safe fallback if no webhook is set
        print("⚠️ No ZAPIER_WEBHOOK_URL set. Returning dummy status 200.")
        return 200

    payload = {
        "text": post_text,
        "scheduled_time": scheduled_time
    }
    try:
        response = requests.post(webhook_url, json=payload)
        return response.status_code
    except Exception as e:
        print("Error calling webhook:", e)
        # Return a fake 'success' code for demo purposes
        return 200


In [7]:
# LeadResearchAgent
class LeadResearchAgent:
    def run(self, lead_name):
        result = search_company_profile(lead_name)
        if result:
            return {
                "name": lead_name,
                "company": result[0].get("title"),
                "url": result[0].get("link"),
                "snippet": result[0].get("snippet")
            }
        # Fallback if search fails
        return {"name": lead_name, "company": None, "url": None, "snippet": None}


# CRMUpdaterAgent
class CRMUpdaterAgent:
    def run(self, lead_profile):
        return update_lead_record(lead_profile)


# EmailGeneratorAgent
from vertexai.preview.generative_models import GenerativeModel

class EmailGeneratorAgent:
    def __init__(self):
        # Model name from Gemini on Vertex AI
        self.model = GenerativeModel("gemini-pro")

    def run(self, lead_profile):
        prompt = f"""
        You are a helpful CRM and marketing assistant.

        Generate a warm, professional follow-up email to a new lead.

        Lead Name: {lead_profile.get('name')}
        Company: {lead_profile.get('company')}
        Website: {lead_profile.get('url')}
        Summary: {lead_profile.get('snippet')}

        The email should:
        - Be friendly but professional
        - Mention the company context
        - Invite them to continue the conversation
        - Be in plain, clear English.
        """
        response = self.model.generate_content(prompt)
        return response.text


# ContentSchedulerAgent
class ContentSchedulerAgent:
    def run(self, post_text, preferred_time="tomorrow 9am"):
        """
        Uses our scheduling tool to 'schedule' a social post.
        In Kaggle this will likely just return status 200 for demo.
        """
        return schedule_social_post(post_text, preferred_time)


In [8]:
def run_full_pipeline(lead_name: str):
    """
    Orchestrates the full CRM/Marketing workflow:
    1. Research lead/company
    2. Update CRM
    3. Generate follow-up email with Gemini
    4. Schedule a social post (or simulate it)
    """

    #  Lead research
    lead_profile = LeadResearchAgent().run(lead_name)

    #  CRM update
    crm_status = CRMUpdaterAgent().run(lead_profile)

    #  Email generation
    email_text = EmailGeneratorAgent().run(lead_profile)

    #  Schedule post (we will just reuse email text as example content)
    scheduled_status = ContentSchedulerAgent().run(email_text)

    # Package results
    return {
        "lead": lead_profile,
        "crm_update": crm_status,
        "email_draft": email_text,
        "scheduled_post": scheduled_status,
    }


In [9]:
# ✅ NEW EmailGeneratorAgent WITHOUT Gemini (no cloud project needed)

class EmailGeneratorAgent:
    def __init__(self):
        pass  # no external model needed

    def run(self, lead_profile):
        name = lead_profile.get("name") or "there"
        company = lead_profile.get("company") or "your company"
        website = lead_profile.get("url") or ""
        snippet = lead_profile.get("snippet") or ""

        email = f"""
Subject: Great connecting with you, {company}

Hi {name},

I hope you're doing well.

I came across {company} and was really interested in what you’re doing{f' – especially {snippet}' if snippet else ''}.
I’d love to connect and explore how I can support your marketing and operations, especially around CRM follow-ups and content.

You can learn more here: {website if website else '[website not available]'}.

If you’re open to it, I’d be happy to schedule a quick call to discuss how we can work together.

Best regards,
Leah
Leah The Digital VA
"""
        return email


In [10]:
# 🔥 DEMO: Run the Full CRM/Marketing Assistant Pipeline

result = run_full_pipeline("Acme Corp")  # You can change this to any company name

print("🔍 LEAD PROFILE:")
print(result["lead"])

print("\n🗃 CRM UPDATE STATUS:")
print(result["crm_update"])

print("\n📬 EMAIL DRAFT FROM GEMINI:\n")
print(result["email_draft"])

print("\n📅 SCHEDULED POST STATUS:")
print(result["scheduled_post"])


⚠️ No ZAPIER_WEBHOOK_URL set. Returning dummy status 200.
🔍 LEAD PROFILE:
{'name': 'Acme Corp', 'company': None, 'url': None, 'snippet': None}

🗃 CRM UPDATE STATUS:
404

📬 EMAIL DRAFT FROM GEMINI:


Subject: Great connecting with you, your company

Hi Acme Corp,

I hope you're doing well.

I came across your company and was really interested in what you’re doing.
I’d love to connect and explore how I can support your marketing and operations, especially around CRM follow-ups and content.

You can learn more here: [website not available].

If you’re open to it, I’d be happy to schedule a quick call to discuss how we can work together.

Best regards,
Leah
Leah The Digital VA


📅 SCHEDULED POST STATUS:
200
